# Machine Learning Exercise 7 - K-Means Clustering & PCA

In this exercise we'll implement K-means clustering and use it to compress an image.  We'll start with a simple 2D data set to see how K-means works, then we'll apply it to image compression.  We'll also experiment with principal component analysis and see how it can be used to find a low-dimensional representation of images of faces.

TODO - Finish this notebook!